In [1]:
import pandas as pd
from datetime import datetime, timedelta
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/javier/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
df = pd.read_csv('2020 - Semana 1 (05012020).txt',error_bad_lines=False,encoding = 'unicode_escape', 
                 header=3, sep='\t', skiprows=[4], skipinitialspace=True, 
                 usecols=['Descripción','Título/Descripción', 'Cadena', 'Fecha', 'Género','Público objetivo', 
                          'Productora','Calificación de Edad','Ind. 4+ (inv)','Amas de Casa (c/inv.)'
                          ,'Ind. 4 - 12 (c/inv.)', 'Ind. 16+ (c/inv.)','Ind. 4+ (inv).1','Amas de Casa (c/inv.).1',
                          'Ind. 4 - 12 (c/inv.).1', 'Ind. 16+ (c/inv.).1'])

In [3]:
#ELIMINAMOS LAS CABECERAS DE LOS PROGRAMAS EN LA COLUMNA DESCRIPCIÓN
df.dropna(inplace = True)
df = df[df.Descripción.str.startswith('<<')]
df.reset_index(drop = True,inplace = True)

In [4]:
#ELIMINAMOS LOS SIMBOLOS <> DEL DATA FRAME
df1 = pd.DataFrame()
for i in df.columns:
    df1 = pd.concat([df1,df.loc[:,str(i)].str.translate({ord(x): None for x in ('><')})], axis = 1)

In [5]:
#DIVIDIMOS LA COLUMNA EN DOS PARA PODER OPERAR SOLO CON LA HORA 
#(NO SE PUEDE PASAR A DATETIME YA QUE NO ESTÁ EN EL RANGO ADECUADO)
df1[['Hora','Minuto']] = df1.Descripción.str.split(':', expand = True)
df1.head(5)

,Descripción,Título/Descripción,Cadena,Fecha,Género,Público objetivo,Productora,Calificación de Edad,Ind. 4+ (inv),Amas de Casa (c/inv.),Ind. 4 - 12 (c/inv.),Ind. 16+ (c/inv.),Ind. 4+ (inv).1,Amas de Casa (c/inv.).1,Ind. 4 - 12 (c/inv.).1,Ind. 16+ (c/inv.).1,Hora,Minuto
0,02:30,FELIZ 2020,La1,01/01/2020,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,764","3,066","0,880","2,992","1245,629","569,778","37,769","1176,382",02,30
1,02:31,FELIZ 2020,La1,01/01/2020,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,925","3,234","1,017","3,161","1318,282","600,972","43,641","1243,164",02,31
2,02:32,FELIZ 2020,La1,01/01/2020,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,924","3,230","0,963","3,167","1317,970","600,309","41,330","1245,162",02,32
3,02:33,FELIZ 2020,La1,01/01/2020,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,845","3,108","0,963","3,076","1282,350","577,546","41,330","1209,543",02,33
4,02:34,FELIZ 2020,La1,01/01/2020,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,623","2,974","0,758","2,844","1182,421","552,688","32,536","1118,408",02,34


In [6]:
#PASAMOS LA COLUMNA FECHA A DATETIME PARA PODER RESTAR DIAS CUANDO SEA NECESARIO
df1['Fecha'] = pd.to_datetime(df1['Fecha'],format = "%d/%m/%Y", dayfirst=True)
df1.dtypes

Descripción                        object
Título/Descripción                 object
Cadena                             object
Fecha                      datetime64[ns]
Género                             object
Público objetivo                   object
Productora                         object
Calificación de Edad               object
Ind. 4+ (inv)                      object
Amas de Casa (c/inv.)              object
Ind. 4 - 12 (c/inv.)               object
Ind. 16+ (c/inv.)                  object
Ind. 4+ (inv).1                    object
Amas de Casa (c/inv.).1            object
Ind. 4 - 12 (c/inv.).1             object
Ind. 16+ (c/inv.).1                object
Hora                               object
Minuto                             object
dtype: object

In [7]:
#SE CAMBIA EL TIPO DE DATO EN LAS COLUMNAS HORA MINUTO PARA PODER TRANSFORMAR A RANGO 00:00-23:00(STR-INT)
df1 = df1.astype({'Hora':'int','Minuto':'int'})
df1.dtypes

Descripción                        object
Título/Descripción                 object
Cadena                             object
Fecha                      datetime64[ns]
Género                             object
Público objetivo                   object
Productora                         object
Calificación de Edad               object
Ind. 4+ (inv)                      object
Amas de Casa (c/inv.)              object
Ind. 4 - 12 (c/inv.)               object
Ind. 16+ (c/inv.)                  object
Ind. 4+ (inv).1                    object
Amas de Casa (c/inv.).1            object
Ind. 4 - 12 (c/inv.).1             object
Ind. 16+ (c/inv.).1                object
Hora                                int64
Minuto                              int64
dtype: object

In [8]:
#SE RESTA DOS HORAS PARA TENER RANGO 00-24
df1.Hora = df1['Hora'].map(lambda value : value - 2)

In [9]:
#SUMAMOS 1 DÍA A LAS HORAS SUPERIORES A 23
ind = df1[df1['Hora']>23].index
df1.loc[ind,'Fecha']= df1.loc[ind,'Fecha'] + timedelta(days=1)

In [10]:
#SE SUSTITUYE EL 24 POR 0 PARA TENER RANGO 00-23
df1.Hora = df1.Hora.replace(24,0)

In [11]:
#CONVIERTO LA FECHA A STRING PARA PODER UNIRLO CON LA HORA
df1['Fecha']=df1.Fecha.apply(lambda x:str(x.date()))
df1[df1['Cadena']=='La1'].head(1413)

,Descripción,Título/Descripción,Cadena,Fecha,Género,Público objetivo,Productora,Calificación de Edad,Ind. 4+ (inv),Amas de Casa (c/inv.),Ind. 4 - 12 (c/inv.),Ind. 16+ (c/inv.),Ind. 4+ (inv).1,Amas de Casa (c/inv.).1,Ind. 4 - 12 (c/inv.).1,Ind. 16+ (c/inv.).1,Hora,Minuto
0,02:30,FELIZ 2020,La1,2020-01-01,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,764","3,066","0,880","2,992","1245,629","569,778","37,769","1176,382",0,30
1,02:31,FELIZ 2020,La1,2020-01-01,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,925","3,234","1,017","3,161","1318,282","600,972","43,641","1243,164",0,31
2,02:32,FELIZ 2020,La1,2020-01-01,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,924","3,230","0,963","3,167","1317,970","600,309","41,330","1245,162",0,32
3,02:33,FELIZ 2020,La1,2020-01-01,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,845","3,108","0,963","3,076","1282,350","577,546","41,330","1209,543",0,33
4,02:34,FELIZ 2020,La1,2020-01-01,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"2,623","2,974","0,758","2,844","1182,421","552,688","32,536","1118,408",0,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,25:57,CINE 2 / LA BUSQUEDA:EL DIARIO SECRETO,La1,2020-01-01,FICCIÓN,FAMILIAR,MULTIPRODUCTORA,Todos los Públicos,"1,416","1,527","0,708","1,524","638,182","283,702","30,373","599,099",23,57
1409,25:58,CINE 2 / LA BUSQUEDA:EL DIARIO SECRETO,La1,2020-01-01,FICCIÓN,FAMILIAR,MULTIPRODUCTORA,Todos los Públicos,"1,358","1,464","0,708","1,457","611,964","272,019","30,373","572,881",23,58
1410,25:59,CINE 2 / LA BUSQUEDA:EL DIARIO SECRETO,La1,2020-01-01,FICCIÓN,FAMILIAR,MULTIPRODUCTORA,Todos los Públicos,"1,194","1,277","0,708","1,275","538,074","237,226","30,373","501,420",23,59
1411,26:00,CINE 2 / LA BUSQUEDA:EL DIARIO SECRETO,La1,2020-01-02,FICCIÓN,FAMILIAR,MULTIPRODUCTORA,Todos los Públicos,"0,850","0,954","0,517","0,902","383,226","177,242","22,183","354,762",0,0


In [12]:
#VOLVEMOS A CONVERTIR HORA Y MINUTO A STR PARA PODER UNIRLOS CON ':' COMO SEPARADOR
df1 = df1.astype({'Hora':'str', 'Minuto':'str'})

In [13]:
#UNIMOS HORA Y MINUTO PARA DESPUÉS UNIRLO A LA FECHA Y CONVERTIRLO EN TIMESTAMP
df1 ['Tiempo'] = df1[['Hora', 'Minuto']].apply(lambda x: ':'.join(x), axis=1)
df1 ['Fecha_comp'] = df1[['Fecha', 'Tiempo']].apply(lambda x: ' '.join(x), axis=1)
df1.Fecha_comp = pd.to_datetime(df1.Fecha_comp)

In [14]:
#ELIMINAMOS LAS COLUMNAS GENERADAS Y USADAS PARA LAS TRANSFORMACIONES
df1 = df1.drop(['Tiempo','Hora', 'Minuto','Descripción','Fecha'], axis = 1)

In [23]:
#CAMBIO COMA POR PUNTO PARA PODER CONVERTIR A ARRAY DE FLOAT
column = ['Ind. 4+ (inv)','Amas de Casa (c/inv.)', 'Ind. 4 - 12 (c/inv.)','Ind. 16+ (c/inv.)',
          'Ind. 4+ (inv).1','Amas de Casa (c/inv.).1','Ind. 4 - 12 (c/inv.).1', 'Ind. 16+ (c/inv.).1']
for i in column:
    df1[i]=df1[i].str.replace(',','.')

In [29]:
#CREO LA VARIABLE TARGET CON LAS COLUMNAS NECESARIAS
df1['Target']= df1[column].apply(lambda x:'/'.join(x),axis=1)
df1['Target']= df1.Target.str.split('/').apply(lambda x:np.array(x,dtype=np.float32))

In [37]:
#ELIMINAMOS LAS VARIABLES UTILIZADAS EN TARGET
df1 = df1.drop(column,axis = 1)

In [39]:
#ELIMINAMOS TITULO/DESCRIPCION DE MOMENTO
df1 = df1.drop('Título/Descripción',axis = 1)

In [42]:
#ORDENO LAS COLUMNAS DE MI DATA FRAME
df1 = df1[['Fecha_comp','Cadena', 'Género', 'Público objetivo', 'Productora',
       'Calificación de Edad', 'Target']]

In [43]:
df1.head(5)

,Fecha_comp,Cadena,Género,Público objetivo,Productora,Calificación de Edad,Target
0,2020-01-01 00:30:00,La1,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"[2.764, 3.066, 0.88, 2.992, 1245.629, 569.778,..."
1,2020-01-01 00:31:00,La1,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"[2.925, 3.234, 1.017, 3.161, 1318.282, 600.972..."
2,2020-01-01 00:32:00,La1,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"[2.924, 3.23, 0.963, 3.167, 1317.97, 600.309, ..."
3,2020-01-01 00:33:00,La1,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"[2.845, 3.108, 0.963, 3.076, 1282.35, 577.546,..."
4,2020-01-01 00:34:00,La1,ENTRETENIMIENTO,FAMILIAR,RTVE,Sin Preaviso,"[2.623, 2.974, 0.758, 2.844, 1182.421, 552.688..."


In [44]:
#CREO EL DATA FRAME CON LAS VARIABLES CATEGÓRICAS
df_cat = df1[['Cadena', 'Género', 'Público objetivo', 'Productora','Calificación de Edad']]

In [50]:
X = df1.drop('Target', axis=1)
y = df1.Target

In [62]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [68]:
cop = pd.get_dummies(X, prefix_sep='_', drop_first=True)

In [66]:
help(make_column_transformer)

Help on function make_column_transformer in module sklearn.compose._column_transformer:

make_column_transformer(*transformers, **kwargs)
    Construct a ColumnTransformer from the given transformers.
    
    This is a shorthand for the ColumnTransformer constructor; it does not
    require, and does not permit, naming the transformers. Instead, they will
    be given names automatically based on their types. It also does not allow
    weighting with ``transformer_weights``.
    
    Read more in the :ref:`User Guide <make_column_transformer>`.
    
    Parameters
    ----------
    *transformers : tuples
        Tuples of the form (transformer, column(s)) specifying the
        transformer objects to be applied to subsets of the data.
    
        transformer : estimator or {'passthrough', 'drop'}
            Estimator must support :term:`fit` and :term:`transform`.
            Special-cased strings 'drop' and 'passthrough' are accepted as
            well, to indicate to drop the co

In [65]:
column_transf = make_column_transformer(OneHotEncoder(),[ 'Género', 'Cadena','Público objetivo', 
                                                         'Productora','Calificación de Edad'],remainder='passthrough')

TypeError: zip argument #1 must support iteration

In [53]:
y

0         [2.764, 3.066, 0.88, 2.992, 1245.629, 569.778,...
1         [2.925, 3.234, 1.017, 3.161, 1318.282, 600.972...
2         [2.924, 3.23, 0.963, 3.167, 1317.97, 600.309, ...
3         [2.845, 3.108, 0.963, 3.076, 1282.35, 577.546,...
4         [2.623, 2.974, 0.758, 2.844, 1182.421, 552.688...
                                ...                        
786929    [0.049, 0.038, 0.0, 0.051, 21.963, 7.098, 0.0,...
786930    [0.049, 0.038, 0.0, 0.051, 21.963, 7.098, 0.0,...
786931    [0.049, 0.038, 0.0, 0.051, 21.963, 7.098, 0.0,...
786932    [0.049, 0.038, 0.0, 0.051, 21.963, 7.098, 0.0,...
786933    [0.067, 0.039, 0.0, 0.076, 29.976, 7.252, 0.0,...
Name: Target, Length: 786934, dtype: object